In [1]:
#Test notebook for tracking rigid body  marker consisting of multiple fiducials within a CT scan volume

#3D data processing
import open3d as o3d
import ct_tracking_library # our helper functions
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import os
#our processing code
#from ct_tracking_library.ct_processing_functions import *

#used for GUI
%matplotlib notebook
from ct_tracking_library import gui
from ct_tracking_library.ct_tracking_functions import *
from ct_tracking_library.ct_processing_functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def display_Dicom(file1,file2):
    assert isinstance(file1,str)
    assert isinstance(file1,str)
    img1 = sitk.ReadImage(file1)
    #img2 = sitk.ReadImage(file2)
    img2 = sitk.GetImageFromArray(file2)
    print_info(img1)
    print_info(img2)
    # Obtain foreground masks for the two images using Otsu thresholding, we use these later on.
    msk1 = sitk.OtsuThreshold(img1,0,1)
    msk2 = sitk.OtsuThreshold(img2,0,1)

    gui.MultiImageDisplay(image_list = [img1, img2],
                          title_list = ['Original', 'Estimated'],
                          figure_size=(9,3))

In [3]:
def print_info(selected_image):
    print('origin: ' + str(selected_image.GetOrigin()))
    print('size: ' + str(selected_image.GetSize()))
    print('spacing: ' + str(selected_image.GetSpacing()))
    print('direction: ' + str(selected_image.GetDirection()))
    print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
    print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))

In [ ]:
with open('./test_data/bb_marker3.pkl', 'rb') as f:
        image_stack, slice_spacing, spacing_x, spacing_y, _, _, _ = pickle.load(f)
display_Dicom("temp_mesh.mha",image_stack)

In [2]:
#convert scan to mesh for finding markers via thresholding
# !!! comment the below code to change the way file is loaded !!!
#convert_scan_to_mesh(scan_file='./test_data/bb_marker3.pkl', debug=True)
convert_scan_to_mesh_mha(scan_file='./test_data/biopsy 4', threshold_value = 1200, debug=True)

spacing:  [1.25     0.976563 0.976563]
image_stack shape:  (37, 512, 512)
Displaying segmented mesh


In [4]:
#choose target marker
target_marker = 'marker1'

#find candidate fiducials within the scanner returning clustered centroids that could be rigid body markers
marker, marker_centroid_coordinates, pcd_selected, mesh, coordinate_frame, good_centroid_clusters = find_candidate_centroids(target_marker = target_marker)

#display candidate fiducials
o3d.visualization.draw_geometries([pcd_selected, mesh])
print('marker centroid coordinates from design: {}'.format(marker))
print('found centroid coordinates: {}'.format(marker_centroid_coordinates))

#find best transform and marker from candidate fiducials
final_R, final_t, permuted_centroids, min_error = find_best_transform_from_candidate_marker_clusters(marker, good_centroid_clusters)

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 44


  0%|          | 0/44 [00:00<?, ?it/s]c:\Users\snowl\Documents\GitHub\ct-fiducial-tracking\ct_tracking_library\ct_tracking_functions.py:376: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  C, residules, rank, singval = np.linalg.lstsq(A,f)
100%|██████████| 44/44 [00:00<00:00, 118.60it/s]


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 1
point cloud has 1 clusters
[0 1 2 3 4]
Multiple good clusters/centroids found
marker centroid coordinates from design: [[  0.           0.           0.        ]
 [ -5.3257001    8.86418433   5.04704152]
 [-13.51012247  20.35648747  13.41037419]
 [ 11.34994372  29.71443298  11.71348397]
 [ 16.07976587  10.72837663   0.39595237]]
found centroid coordinates: [ 15.31429282 327.55045063  90.38617847]
[0.03503577]
Everything looks good!
the final error is:  0.035035773919779876


In [5]:
#display tracked  markers and coordinate frames
marker = np.load('./test_data/'+target_marker+'.npy')
marker_3d_base, marker_3d_transformed, err = visualize_tracked_marker(marker, final_R, final_t, permuted_centroids)
o3d.visualization.draw_geometries(marker_3d_base+marker_3d_transformed)

#tracking error from final transform
np.set_printoptions(2)
print(err)
print(np.linalg.norm(err))

#visualize everything at once
visualization_list = [pcd_selected, mesh]+marker_3d_base+marker_3d_transformed
o3d.visualization.draw_geometries(visualization_list)

[[ 0.01  0.   -0.05]
 [ 0.02 -0.    0.02]
 [ 0.01  0.    0.03]
 [-0.02  0.    0.02]
 [-0.02 -0.   -0.02]]
0.07834237212894206
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
